# REDPy EQ Rate Plot

In [ ]:
import redpy.config
import redpy.table
import redpy.plotting
#import numpy as np


In [ ]:
from tables import *
import numpy as np
import matplotlib
import datetime
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.dates
import time
import pandas as pd
import redpy.cluster
import redpy.correlation
from redpy.optics import *
import os
import shutil
import glob
import urllib
from obspy import UTCDateTime
from obspy.geodetics import locations2degrees
from obspy.taup import TauPyModel
import pandas as pd
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from matplotlib.transforms import offset_copy
from bokeh.plotting import figure, output_file, save, gridplot
from bokeh.models import HoverTool, ColumnDataSource, OpenURL, TapTool, Range1d, Div, Span
from bokeh.models import Arrow, VeeHead, ColorBar, LogColorMapper, LogTicker, LabelSet
from bokeh.models.glyphs import Line, Quad
from bokeh.layouts import column
from bokeh.palettes import inferno, all_palettes
import urllib.request

In [ ]:
config = '/Users/jaywellik/Development/REDPy_svgfigures/agung_grlB2.cfg'
opt = redpy.config.Options(config)
h5file, rtable, otable, ttable, ctable, jtable, dtable, ftable = redpy.table.openTable(opt)


In [ ]:
dt = rtable.cols.startTimeMPL[:]
ot = otable.cols.startTimeMPL[:]
fi = np.nanmean(rtable.cols.FI[:], axis=1)
fio = np.nanmean(otable.cols.FI[:], axis=1)
longevity = ftable.cols.longevity[:]
famstarts = ftable.cols.startTime[:]
alltrigs = ttable.cols.startTimeMPL[:]
trigrange = [ttable.cols.startTimeMPL[0], ttable.cols.startTimeMPL[-1]]
    
# Read in annotation file (if it exists)
if opt.anotfile != '':
    df = pd.read_csv(opt.anotfile)
    
# Create histogram of events/dybin
histT, hT = np.histogram(alltrigs, bins=np.arange(min(alltrigs),
    max(alltrigs+opt.dybin), opt.dybin))
histR, hR = np.histogram(dt, bins=np.arange(min(alltrigs),
    max(alltrigs+opt.dybin), opt.dybin))
        
# Determine padding for hover bars (~1% of window range on each side)
barpad = (max(alltrigs)-min(alltrigs))*0.01
barpadr = opt.recplot*0.01
        
# Create histogram of events/hrbin
histTr, hTr = np.histogram(alltrigs, bins=np.arange(max(alltrigs)-opt.recplot,
    max(alltrigs+opt.hrbin/24), opt.hrbin/24))
histRr, hRr = np.histogram(dt, bins=np.arange(max(alltrigs)-opt.recplot,
    max(alltrigs+opt.hrbin/24), opt.hrbin/24))

## bokeh EQ rate

In [ ]:

oTOOLS = ['pan,box_zoom,reset,save,tap']


o1 = figure(tools=oTOOLS, plot_width=1250, plot_height=250, x_axis_type='datetime',
    #x_range=o0.x_range,
    title='Frequency Index')
o1.grid.grid_line_alpha = 0.3
o1.xaxis.axis_label = 'Date'
o1.yaxis.axis_label = 'FI'
    
if opt.anotfile != '':
    for row in df.itertuples():
        spantime = (datetime.datetime.strptime(row[1]
            ,'%Y-%m-%dT%H:%M:%S')-datetime.datetime(1970, 1, 1)).total_seconds()
        o1.add_layout(Span(location=spantime*1000, dimension='height',
            line_color=row[2], line_width=row[3], line_dash=row[4],
            line_alpha=row[5]))
o1.circle(matplotlib.dates.num2date(dt), fi, color='red', line_alpha=0,
    size=3, fill_alpha=0.5)
        
o1r = figure(tools=oTOOLS, plot_width=1250, plot_height=250, x_axis_type='datetime',
    #x_range=o0r.x_range,
    title='Frequency Index')
o1r.grid.grid_line_alpha = 0.3
o1r.xaxis.axis_label = 'Date'
o1r.yaxis.axis_label = 'FI'
    
if opt.anotfile != '':
    for row in df.itertuples():
        spantime = (datetime.datetime.strptime(row[1]
            ,'%Y-%m-%dT%H:%M:%S')-datetime.datetime(1970, 1, 1)).total_seconds()
        o1r.add_layout(Span(location=spantime*1000, dimension='height',
            line_color=row[2], line_width=row[3], line_dash=row[4],
            line_alpha=row[5]))
# Put invisible points in for case that there are no events
o1r.circle(matplotlib.dates.num2date(hTr[0:2]), [1, 1], line_alpha=0, fill_alpha=0)
o1r.circle(matplotlib.dates.num2date(dt[dt>(max(alltrigs)-opt.recplot)]),
    fi[dt>(max(alltrigs)-opt.recplot)], color='red', line_alpha=0,
    size=3, fill_alpha=0.5)


In [ ]:
from bokeh.io import show
show(o1)

## matplotlib EQ rates

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.collections import EventCollection
import numpy as np
import matplotlib.dates as mdates


fig = plt.figure(figsize=[6.4,2], tight_layout=True)
ax = fig.add_subplot(1,1,1)

x1 = matplotlib.dates.num2date(ot)
x2 = matplotlib.dates.num2date(dt)
y1 = fio
y2 = fi
#ax.plot(x1,y1, 'ko', label='Orphans', alpha=0.5)
#ax.plot(x2,y2, 'ro', label='Repeaters', alpha=0.5)
ax.scatter(x1,y1, c='k', marker='o', s=10, label='Orphans', alpha=0.5)
ax.scatter(x2,y2, c='r', marker='o', s=10, label='Repeaters', alpha=0.5)

plt.xlabel('Date', fontname='Times', fontsize=10)
plt.ylabel('FI', fontname='Times', fontsize=10)
plt.title('Frequency Index', fontsize=10)
ax.tick_params(axis='both', which='major', labelsize=10)
# ax.xaxis.label.set_size(28)
plt.yticks([-1,0,1])
ax.set_xlim(trigrange)
ax.set_ylim([-1,1])
ax.legend()

import pandas as pd
drng = pd.date_range(start='2017/11/1', end='2018/1/15', freq='MS')
xticks = drng.to_pydatetime()
xtick_labels = list(drng.strftime('%Y %b'))
plt.xticks(xticks, xtick_labels)

fig.show()

In [ ]:
fig.savefig('/Users/jaywellik/Desktop/TestFigure_FI.png', dpi=450, facecolor=None, edgecolor=None,
        orientation='portrait', papertype=None,
        transparent=True, bbox_inches=None, pad_inches=0,
        frameon=None, metadata=None)